In [1]:
import json
import requests

def fetch_miners_data(block_heights, days=30):
    """
    Fetch miners data from the API.
    
    Args:
        block_heights (list): List of block heights to query
        days (int, optional): Number of days to consider for the average. Defaults to 30.
    
    Returns:
        dict: JSON response from the API
    """
    base_url = "http://5.78.102.130:8000/sigscore/miners/average-participation"
    
    # Convert block heights list to comma-separated string
    blocks_param = ",".join(map(str, block_heights))
    
    # Make the API request with both blocks and days parameters
    response = requests.get(f"{base_url}?blocks={blocks_param}&days={days}")
    response.raise_for_status()  # Raise exception for bad status codes
    
    return response.json()

def validate_percentages(miners_data):
    """
    Validate that participation percentages sum to approximately 100%.
    
    Args:
        miners_data (dict): JSON data containing miner information
    
    Returns:
        bool: True if percentages are valid, False otherwise
    """
    total_percentage = sum(miner['avg_participation_percentage'] for miner in miners_data['miners'])
    if not (99.99 <= total_percentage <= 100.01):  # Allow for small floating-point errors
        print(f"Warning: Percentages sum to {total_percentage}%, not 100%")
        return False
    return True

def generate_distribution(miners_data, total_amount, token_name):
    """
    Generate a distribution JSON based on normalized participation percentages.
    """
    # First validate and print the original percentage sum
    total_percentage = sum(miner['avg_participation_percentage'] for miner in miners_data['miners'])
    print(f"Original percentage sum: {total_percentage}%")
    
    recipients = []
    running_total = 0
    
    # First pass: calculate amounts with normalized percentages
    for miner in miners_data['miners']:
        # Normalize the percentage
        normalized_percentage = (miner['avg_participation_percentage'] / total_percentage)
        amount = round(total_amount * normalized_percentage, 8)
        if amount > 0:
            recipients.append({
                "address": miner['miner_address'],
                "amount": amount
            })
            running_total += amount
    
    # Adjust for any rounding discrepancy
    if recipients:
        discrepancy = round(total_amount - running_total, 8)
        if discrepancy != 0:
            # Add the discrepancy to the largest recipient to maintain the total
            largest_recipient = max(recipients, key=lambda x: x['amount'])
            largest_recipient['amount'] = round(largest_recipient['amount'] + discrepancy, 8)
    
    # Verify final total
    final_total = sum(recipient['amount'] for recipient in recipients)
    print(f"Final distribution total: {final_total}")
    
    return {
        "distributions": [
            {
                "token_name": token_name,
                "recipients": recipients
            }
        ]
    }

def save_distribution_json(distribution, output_file):
    """
    Save distribution data to a JSON file.
    
    Args:
        distribution (dict): Distribution data to save
        output_file (str): Path to the output JSON file
    """
    with open(output_file, 'w') as f:
        json.dump(distribution, f, indent=2)



In [3]:
def main():
    # Example block heights from your query
    block_heights = [1499245, 1499174, 1499031, 1498794, 1498726, 1498533, 1498515, 1498498,
                  1498318, 1498095, 1497930, 1497877, 1497861, 1497699, 1497628, 1497566,
                    1497566, 1497566, 1497566, 1495958, 1495702, 1495300, 1495201, 1495133,
                      1494969, 1494873, 1494829, 1494642, 1494502, 1494175]
    try:
        # Fetch data from API
        miners_data = fetch_miners_data(block_heights)
        
        # Generate distribution with example values
        total_amount = 67  # Example amount
        token_name = "ergo"  # Example token name
        
        result = generate_distribution(miners_data, total_amount, token_name)
        
        # Pretty print the result to console
        print(json.dumps(result, indent=2))
        
        # Save to JSON file
        output_file = "distribution_output.json"
        save_distribution_json(result, output_file)
        print(f"\nDistribution data saved to {output_file}")
        
    except requests.RequestException as e:
        print(f"Error fetching data from API: {e}")
    except Exception as e:
        print(f"Error processing data: {e}")

if __name__ == "__main__":
    main()

Original percentage sum: 104.6589%
Final distribution total: 67.0
{
  "distributions": [
    {
      "token_name": "ergo",
      "recipients": [
        {
          "address": "9f6PPoCYrvHQbS6U8s7y6wE1S6pFET712ajBCzH5Ep7mYTGBudy",
          "amount": 33.58844014
        },
        {
          "address": "9eYN6LACnheBuPYPMjhoERN5gjw2NE7mTfri378gZPazcoEPGpL",
          "amount": 2.52228907
        },
        {
          "address": "9i4DWcdw6c4PMdSHvBRJs2aPQ5rqzd4R4Vf1JPxeVi7wsZVd4c1",
          "amount": 2.48784767
        },
        {
          "address": "9h6bjuzth7XDNeBxwFjpYSfmUkfBiepE8MGJcvmsVkeCtGYYYAb",
          "amount": 1.80023581
        },
        {
          "address": "9fUBHUXfduFkCvHcx1eMV799a8uUp3wxYh4WkNNLwhLtURmPMWh",
          "amount": 1.52560079
        },
        {
          "address": "9fNJdGEeXiunSW4tFXG2jKhHrH26Z5RHMCsJw59ko1VqwqFioSH",
          "amount": 1.43700058
        },
        {
          "address": "9hSk13XDU8keWAp3KxKSgUVCkfDcw577jAjM6SgbRpuYeL8ygPH",


In [8]:
import os
import requests
from typing import List
import json

class BlockHeightCollector:
    def __init__(self, wallet_address: str = "9fE5o7913CKKe6wvNgM11vULjTuKiopPcvCaj7t2zcJWXM2gcLu"):
        self.api_base = "https://api.ergoplatform.com/api/v1"
        self.wallet_address = os.getenv('WALLET_ADDRESS', wallet_address)

    def get_latest_outgoing_block(self) -> int:
        try:
            response = requests.get(
                f"{self.api_base}/addresses/{self.wallet_address}/transactions",
                params={"offset": 0, "limit": 50}
            )
            response.raise_for_status()
            
            for tx in response.json()["items"]:
                if any(input_data.get("address") == self.wallet_address 
                       for input_data in tx.get("inputs", [])):
                    return tx["inclusionHeight"]
            return 0
        except Exception as e:
            print(f"Error finding latest outgoing transaction: {e}")
            return 0

    def get_blocks_since_last_outgoing(self) -> List[int]:
        try:
            min_block = self.get_latest_outgoing_block()
            min_block = 1449260
            
            if not min_block:
                return []

            all_blocks = set()
            current_offset = 0
            limit = 100
            
            while True:
                response = requests.get(
                    f"{self.api_base}/addresses/{self.wallet_address}/transactions",
                    params={
                        "offset": current_offset,
                        "limit": limit,
                        "concise": True
                    }
                )
                response.raise_for_status()
                data = response.json()
                
                if not data["items"]:
                    break
                    
                for tx in data["items"]:
                    if tx.get("inclusionHeight", 0) > min_block:
                        all_blocks.add(tx["inclusionHeight"])
                        
                if len(data["items"]) < limit:
                    break
                    
                current_offset += limit
                
            return sorted(list(all_blocks))
        except Exception as e:
            print(f"Error fetching blocks: {e}")
            return []

def main():
    collector = BlockHeightCollector()
    block_heights = collector.get_blocks_since_last_outgoing()
    
    if not block_heights:
        print("No blocks found since last outgoing transaction")
        return
    
    try:
        miners_data = fetch_miners_data(block_heights)
        distribution = generate_distribution(miners_data, 5.5, "ergo")
        output_file = f"distribution_{len(block_heights)}_blocks.json"
        save_distribution_json(distribution, output_file)
        print(f"Distribution saved to {output_file}")
        
    except Exception as e:
        print(f"Error processing distribution: {e}")

if __name__ == "__main__":
    main()

Error processing distribution: 500 Server Error: Internal Server Error for url: http://5.78.102.130:8000/sigscore/miners/average-participation?blocks=1453434,1453711,1455039,1455066,1455075,1455101,1455651,1455657,1455660,1455665,1456421,1456437,1456438,1456442,1456455,1456464,1458634,1463412,1468646,1474187,1478414,1478591,1487742
